In [1]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib import style
import glob
%matplotlib inline
style.use('ggplot')

In [2]:
tickers = pd.read_csv('tickers_new.csv')
ticker_list = tickers['ticker'].values.tolist()
company_list = tickers['company'].values.tolist()
sector_list = tickers['sector'].values.tolist() 

#### Plug in both company name and sector for each company ticket

In [3]:
Y = []
for symbols,company,sector in zip(ticker_list,company_list,sector_list):
    df = pd.read_csv(f'stock_csvs/stock_pup_{symbols}.csv')
    df['symbol'] = symbols
    df['company'] = company
    df['sector']  = sector
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    Y.append(df)

In [4]:
fund_stocks = pd.concat(Y, sort = False)

In [5]:
fund_stocks.shape

(62850, 44)

#### Quarter End to Date Time

In [6]:
# Quarter End to Date time
fund_stocks['Quarter end'] = pd.to_datetime(fund_stocks['Quarter end'])
fund_stocks.set_index("Quarter end", inplace=True)

#### Change date from 2000 to current year

In [7]:
#2000 to current year
five_yr_fstock = fund_stocks['2000':]

In [8]:
#convert strings into numeric integers
five_yr_fstock = five_yr_fstock.apply(pd.to_numeric, errors='ignore')

#### create unmatched companies in new dataframe

In [9]:
### assign variable to unmatch
nomatch_companies = five_yr_fstock[five_yr_fstock['company'] == 'nomatch']

#### Create a new data frame with nomatched companies

In [10]:
## assign variable to matched company and sector
company_fund = five_yr_fstock[five_yr_fstock['company'] != 'nomatch']

In [11]:
# 135 companies were not matched to finviz company
nomatch_companies.groupby('symbol')['company'].nunique().value_counts()

1    189
Name: company, dtype: int64

In [12]:
# 566 companies were matched to finviz company
company_fund.groupby('company')['symbol'].nunique().value_counts()

1    566
Name: symbol, dtype: int64

#### replace all 0's and None to nan's

In [13]:
company_fund = company_fund.replace(to_replace='0', value= np.nan)

In [14]:
company_fund = company_fund.replace(to_replace='None', value= np.nan )

In [15]:
company_fund = company_fund.apply(pd.to_numeric, errors='ignore')

In [16]:
company_fund['P/E ratio'].fillna(0, inplace=True)

In [17]:
company_fund.fillna(0, inplace = True)

#### Assigning reported_pe for approiate companies

In [18]:
company_fund['reported_pe'] = company_fund['P/E ratio'].apply(lambda x: 1 if x != 0 else 0)

In [19]:
company_fund['reported_pe'].value_counts()

1    35178
0     5388
Name: reported_pe, dtype: int64

#### Assigning reported_earnings for appropiate companies

In [20]:
company_fund['reported_earnings'] = company_fund['Earnings'].apply(lambda x: 1 if x != 0 else 0)

In [21]:
company_fund['reported_earnings'].value_counts()

1    40414
0      152
Name: reported_earnings, dtype: int64

In [38]:
company_fund['growth'] = company_fund['P/E ratio'].apply(lambda x: 1 if x == 0 or x >= 25 else 0)

In [40]:
company_fund['growth'].value_counts()

0    24024
1    16542
Name: growth, dtype: int64

In [110]:
company_fund[company_fund['symbol'] == 'ARRS']

,Quarter end,symbol,company,sector,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,...,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,reported_pe,reported_earnings,growth
4224,2018-12-31,ARRS,ARRIS International plc,Technology,174116131,174116131,1.0,7.327869e+09,3.112639e+09,4.322021e+09,...,0.0,0.6793,0.4083,0.0169,0.92,1.35,1.5818,1,1,1
4225,2018-09-30,ARRS,ARRIS International plc,Technology,173328693,173328693,1.0,7.186605e+09,2.843385e+09,4.233095e+09,...,0.0,0.6987,0.4089,0.0122,0.91,1.29,1.5381,1,1,1
4226,2018-06-30,ARRS,ARRIS International plc,Technology,180330719,180330719,1.0,7.401524e+09,2.973463e+09,4.292220e+09,...,0.0,0.6706,0.4179,0.0182,0.90,0.61,1.5855,1,1,1
4227,2018-03-31,ARRS,ARRIS International plc,Technology,186263077,186263077,1.0,7.366488e+09,2.832112e+09,4.209114e+09,...,0.0,0.6670,0.4265,0.0175,0.89,0.49,1.6516,1,1,1
4228,2017-12-31,ARRS,ARRIS International plc,Technology,185238691,185238691,1.0,7.624257e+09,2.913540e+09,4.440223e+09,...,0.0,0.6679,0.4156,0.0139,0.88,-0.50,1.5071,1,1,1
4229,2017-09-30,ARRS,ARRIS International plc,Technology,187021231,187021231,1.0,7.664287e+09,3.569462e+09,4.437018e+09,...,0.0,0.6620,0.4164,0.0253,0.88,0.48,1.8279,1,1,1
4230,2017-06-30,ARRS,ARRIS International plc,Technology,187536426,187536426,1.0,7.510535e+09,3.334857e+09,4.368462e+09,...,0.0,0.6876,0.4133,0.0193,0.88,1.20,1.7926,1,1,1
4231,2017-03-31,ARRS,ARRIS International plc,Technology,187781876,187781876,1.0,7.354731e+09,3.082491e+09,4.225726e+09,...,0.0,0.6981,0.4205,0.0271,0.88,1.21,1.8252,1,1,1
4232,2016-12-31,ARRS,ARRIS International plc,Technology,190227443,190227443,1.0,7.758362e+09,3.280072e+09,4.531625e+09,...,0.0,0.6836,0.4110,0.0027,0.90,0.04,1.7883,0,1,1
4233,2016-09-30,ARRS,ARRIS International plc,Technology,189684048,189684048,1.0,7.651816e+09,3.049712e+09,4.591412e+09,...,0.0,0.7286,0.3947,0.0000,0.91,1.43,1.7396,0,1,1


####  Categorize Sectors

In [41]:
company_fund['sector'].value_counts().unique

<bound method Series.unique of Services            7989
Financial           7601
Technology          5353
Consumer Goods      5072
Industrial Goods    4302
Basic Materials     4239
Healthcare          4065
Utilities           1945
Name: sector, dtype: int64>

In [24]:
company_fund = company_fund.reset_index()
sectors = pd.get_dummies(company_fund['sector'], prefix= 'sector')
companies = company_fund.merge(sectors, left_index=True , right_index= True)

In [28]:
companies.set_index("Quarter end", inplace=True)

In [129]:
companies.to_csv('/Users/flatironschool/UrPortfolio/companies.csv', index = True)

### Check Company Health And Growth Fundamentals

In [115]:
companies[['symbol','health_dtbr']]

,symbol,health_dtbr
Quarter end,,
2019-04-30,A,1
2019-01-31,A,1
2018-10-31,A,1
2018-07-31,A,1
2018-04-30,A,1
2018-01-31,A,1
2017-10-31,A,1
2017-07-31,A,1
2017-04-30,A,1


In [116]:
companies['Assets']/companies['Long-term debt']

Quarter end
2019-04-30    5.017798
2019-01-31    4.978865
2018-10-31    4.747638
2018-07-31    4.640912
2018-04-30    4.880000
2018-01-31    4.832222
2017-10-31    4.678512
2017-07-31    4.586896
2017-04-30    4.448391
2017-01-31    4.368479
2016-10-31    4.080544
2016-07-31    4.681598
2016-04-30    4.619105
2016-01-31    4.417423
2015-10-31    4.519033
2015-07-31    4.381269
2015-04-30    4.475845
2015-01-31    4.534982
2014-10-31    3.921434
2014-07-31    4.759285
2014-04-30    4.094319
2014-01-31    3.947310
2013-10-31    3.959244
2013-07-31    3.805257
2013-04-30    5.027066
2013-01-31    5.046423
2012-10-31    4.988636
2012-07-31    5.692532
2012-04-30    4.887331
2012-01-31    4.716952
                ...   
2001-03-31         inf
2000-12-31         inf
2000-09-30         inf
2000-06-30         inf
2000-03-31         inf
2019-03-31    1.654454
2018-12-31    1.672668
2018-09-30    1.623816
2018-06-30    1.736226
2018-03-31    1.754138
2017-12-31    1.733495
2017-10-01    1.875606

In [74]:
companies['health_cr'] = companies['Current ratio'].apply(lambda x: 1 if x >= 1.5 and x <= 3.0 else 0)

In [75]:
companies['health_dtbr'] = companies['Long-term debt to equity ratio'].apply(lambda x: 1 if x >= .05 else 0)

In [76]:
companies['growth_roa'] = companies['ROA'].apply(lambda x: 1 if x >= .05 else 0)

In [77]:
companies['growth_roe'] = companies['ROE'].apply(lambda x: 1 if x >= .1 else 0)

In [78]:
companies['health_dyp'] = companies['Dividend payout ratio'].apply(lambda x: 1 if x >= .55 else 0)

In [81]:
companies.to_csv('/Users/flatironschool/UrPortfolio/companies.csv', index = True)